In [ ]:
prefix = 'robot_1_'
floating_base_IOs = f'{prefix}IOs'
alpha_axis_a = f'{prefix}_axis_a'
alpha_axis_b = f'{prefix}_axis_b'
alpha_axis_c = f'{prefix}_axis_c'
alpha_axis_d = f'{prefix}_axis_d'
alpha_axis_e = f'{prefix}_axis_e'


class Axis_Interface_names:
    manipulator_position = 'position'
    manipulator_filtered_position = 'filtered_position'
    manipulator_velocity = 'velocity'
    manipulator_filtered_velocity = 'filtered_velocity'
    manipulator_estimation_acceleration = "estimated_acceleration"
    manipulator_effort = 'effort'
    
    floating_base_x = 'position.x'
    floating_base_y = 'position.y'
    floating_base_z = 'position.z'

    floating_base_roll = 'roll'
    floating_base_pitch = 'pitch'
    floating_base_yaw = 'yaw'

    floating_dx = 'velocity.x'
    floating_dy = 'velocity.y'
    floating_dz = 'velocity.z'

    floating_roll_vel = 'angular_velocity.x'
    floating_pitch_vel = 'angular_velocity.y'
    floating_yaw_vel = 'angular_velocity.z'

    floating_force_x = 'force.x'
    floating_force_y = 'force.y'
    floating_force_z = 'force.z'
    floating_torque_x = 'torque.x'
    floating_torque_y = 'torque.y'
    floating_torque_z = 'torque.z'

    sim_time = 'sim_time'
    sim_period = 'sim_period'

    imu_roll = "imu_roll"
    imu_pitch = "imu_pitch"
    imu_yaw = "imu_yaw"

    imu_roll_unwrap = "imu_roll_unwrap"
    imu_pitch_unwrap = "imu_pitch_unwrap"
    imu_yaw_unwrap = "imu_yaw_unwrap"

    imu_q_w = "imu_orientation_w"
    imu_q_x = "imu_orientation_x"
    imu_q_y = "imu_orientation_y"
    imu_q_z = "imu_orientation_z"

    imu_wx = "imu_angular_vel_x"
    imu_wy = "imu_angular_vel_y"
    imu_wz = "imu_angular_vel_z"

    imu_ax = "imu_linear_acceleration_x"
    imu_ay = "imu_linear_acceleration_y"
    imu_az = "imu_linear_acceleration_z"

    depth_pressure2 = "depth_from_pressure2"

    dvl_roll = "dvl_gyro_roll"
    dvl_pitch = "dvl_gyro_pitch"
    dvl_yaw = "dvl_gyro_yaw"

    dvl_speed_x = "dvl_speed_x"
    dvl_speed_y = "dvl_speed_y"
    dvl_speed_z = "dvl_speed_z"
    
joints = [alpha_axis_e, alpha_axis_d, alpha_axis_c, alpha_axis_b, alpha_axis_a]

sensors = [
        Axis_Interface_names.imu_roll,
        Axis_Interface_names.imu_pitch,
        Axis_Interface_names.imu_yaw,
        Axis_Interface_names.imu_roll_unwrap,
        Axis_Interface_names.imu_pitch_unwrap,
        Axis_Interface_names.imu_yaw_unwrap,
        Axis_Interface_names.imu_q_w,
        Axis_Interface_names.imu_q_x,
        Axis_Interface_names.imu_q_y,
        Axis_Interface_names.imu_q_z,
        Axis_Interface_names.imu_wx,
        Axis_Interface_names.imu_wy,
        Axis_Interface_names.imu_wz,
        Axis_Interface_names.imu_ax,
        Axis_Interface_names.imu_ay,
        Axis_Interface_names.imu_az,
        Axis_Interface_names.depth_pressure2,
        Axis_Interface_names.dvl_roll,
        Axis_Interface_names.dvl_pitch,
        Axis_Interface_names.dvl_yaw,
        Axis_Interface_names.dvl_speed_x,
        Axis_Interface_names.dvl_speed_y,
        Axis_Interface_names.dvl_speed_z
        ]
prediction_interfaces = [
    "position.x", "position.y", "position.z", "roll", "pitch", "yaw",
    "orientation.w", "orientation.x", "orientation.y", "orientation.z", 
    "velocity.x", "velocity.y", "velocity.z", 
    "angular_velocity.x", "angular_velocity.y", "angular_velocity.z",
]

state_estimate_interfaces = [
    "position_estimate.x", "position_estimate.y", "position_estimate.z",
    "roll_estimate", "pitch_estimate", "yaw_estimate",
    "orientation_estimate.w", "orientation_estimate.x", "orientation_estimate.y", "orientation_estimate.z",
    "velocity_estimate.x", "velocity_estimate.y", "velocity_estimate.z",
    "angular_velocity_estimate.x", "angular_velocity_estimate.y", "angular_velocity_estimate.z",
    "linear_acceleration.x", "linear_acceleration.y", "linear_acceleration.z",
    "angular_acceleration.x", "angular_acceleration.y", "angular_acceleration.z",
    "P_x_x", "P_y_y", "P_z_z", "P_roll_roll", "P_pitch_pitch", "P_yaw_yaw",
    "P_u_u", "P_v_v", "P_w_w", "P_p_p", "P_q_q", "P_r_r",
]

payload_state_interfaces = ["payload.mass", "payload.Ixx", "payload.Iyy", "payload.Izz"]

In [16]:
def get_interface_value(msg, dof_names: list, interface_names: list):
    names = msg.joint_names
    return [
        msg.interface_values[names.index(joint_name)].values[
            msg.interface_values[names.index(joint_name)].interface_names.index(interface_name)
        ]
        for joint_name, interface_name in zip(dof_names, interface_names)
    ]

In [1]:
from mcap.reader import make_reader
from mcap_ros2.decoder import DecoderFactory

rows = []
with open("my_bag_20251220_182123_0.mcap", "rb") as f:
    reader = make_reader(f, decoder_factories=[DecoderFactory()])
    for schema, channel, message, ros_msg in reader.iter_decoded_messages():
        
        ned_pose = get_interface_value(
            ros_msg,
            [floating_base_IOs] * 6,
            [
                Axis_Interface_names.floating_base_x,
                Axis_Interface_names.floating_base_y,
                Axis_Interface_names.floating_base_z,
                Axis_Interface_names.floating_base_roll,
                Axis_Interface_names.floating_base_pitch,
                Axis_Interface_names.floating_base_yaw
            ]
        )
        rows.append({
            "topic": channel.topic,
            "stamp_sec": ros_msg.header.stamp.sec,
            "ned_x": ned_pose[0],
            "ned_y": ned_pose[1],
            "ned_z": ned_pose[2],
            "roll": ned_pose[3],
            "pitch": ned_pose[4],
            "yaw": ned_pose[5],
        })

# print(rows[0])

FileNotFoundError: [Errno 2] No such file or directory: 'my_bag_20251220_182123_0.mcap'

In [3]:
import pandas as pd

df = pd.DataFrame(rows)
